## Cencepts from paper

"Measuring Semantic Similarity between Concepts in Visual Domain" Table 1

In [1]:
concepts = [
    "bay",
    "beach",
    "birds",
    "boeing",
    "buildings",
    "city",
    "clouds",
    "face",
    "f-16",
    "helicopter",
    "mountain",
    "sky",
    "ships",
    "sunset",
    "sunrise",
    "ocean"
]

In [2]:
len(concepts)

16

In [9]:
import json
import time

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains


In [12]:
chrome_options = Options()
chrome_options.binary_location = '/usr/bin/google-chrome-stable'
chrome_options.add_argument("--headless")

# chrome_options.add_argument('--disable-gpu')

# fix for chrome not reachable
# chrome_options.add_argument('--no-sandbox')

### Bing

In [15]:
URL_TEMPLATE = "http://www.bing.com/images/search?q={}&FORM=IGRE"

In [16]:
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="/usr/local/bin/chromedriver")
driver.maximize_window()

In [17]:
def find_index(atags, hval):
    for i in range(len(atags)):
        if atags[i].get_attribute('h') == hval:
            return i
    return -1

In [18]:
def get_url(atag):
    return json.loads(atag.get_attribute("m"))['murl']


def scroll_to(elem):
    ActionChains(driver).move_to_element(elem).perform()
    
def find_all_image_atags():
    return driver.find_elements_by_class_name("iusc")

In [19]:
def get_urls(atags, after_hval = None):
    begin = 0
    res = []
    if after_hval != None:
        begin = find_index(atags, after_hval)+1
        if begin == 0:
            raise Exception("No hval found: {}".format(after_hval))
    for i in range(begin, len(atags)):
        res.append(get_url(atags[i]))
    lasthval = atags[len(atags)-1].get_attribute('h')
    return (lasthval, res)

In [20]:
from selenium.common.exceptions import NoSuchElementException

def collect_all_urls(limit=100):
    res_list = []
    last_hval = None
    loop_limit = limit
    loop_count = 0
    while len(res_list) < limit:
        atags = find_all_image_atags()
        last_hval_cand, urls = get_urls(atags, last_hval)
        if len(urls) == 0:
            try :
                driver.find_element_by_partial_link_text('See more images').click()
            except NoSuchElementException:
                print("reach to last image")
                return res_list
            time.sleep(3)
            atags = find_all_image_atags()
            last_hval_cand, urls = get_urls(atags, last_hval)
            if len(urls) == 0:
                print("No new image urls. Somehing wrong happens")
                return res_list
        last_hval = last_hval_cand
        res_list.extend(urls)
        scroll_to(atags[len(atags)-1])
        if loop_count > loop_limit:
            print("Too much loop, something wrong happens.")
            return res_list
        loop_count += 1
        print(len(res_list))
        time.sleep(1)
    return res_list


In [21]:
def collect_1100_images(cat):
    top_url = URL_TEMPLATE.format(cat)
    driver.get(top_url)
    time.sleep(1)
    return collect_all_urls(1100)

In [3]:
def save_urls(urls, category):
    with open("urls/{}.txt".format(category), "w", encoding='utf8') as f:
        for url in urls:
            f.write(url+"\n")

In [29]:
for cat in concepts:
    urls = collect_1100_images(cat)
    save_urls(urls, cat)

70
105
140
175
210
245
280
315
350
385
420
455
490
525
560
595
630
665
700
735
770
805
840
875
910
945
980
984
reach to last image
35
70
105
140
175
210
245
280
315
350
385
420
455
490
525
560
595
630
665
700
735
770
805
840
875
910
945
980
1000
reach to last image
35
70
105
140
175
210
245
280
315
350
385
420
455
490
525
560
595
630
665
700
735
770
805
840
875
910
945
980
991
reach to last image
35
70
105
140
175
210
245
280
315
350
385
420
455
490
525
560
595
630
665
700
718
reach to last image
35
70
105
140
175
210
245
280
315
350
385
420
455
490
525
560
595
630
665
700
735
770
805
840
870
reach to last image
35
70
105
140
175
210
245
280
315
350
385
420
455
490
525
560
595
630
665
700
735
770
805
840
866
reach to last image
35
70
105
140
175
210
245
280
315
350
385
420
455
490
525
560
595
630
665
700
735
770
805
reach to last image
35
70
105
140
175
210
245
280
315
350
385
420
455
490
525
560
595
630
665
700
735
770
805
840
875
910
945
980
984
reach to last image
35
70
105
140
175


### Remove dup

In [4]:
def read_urls(cat):
    with open("urls/{}.txt".format(cat), encoding='utf8') as f:
        lines = f.readlines()
    return [line.rsplit('\n')[0] for line in lines]

In [10]:
# keep order and remove dup
def uniq_list(urllist):
    done = set()
    ret = []
    for url in urllist:
        if not url in done:
            ret.append(url)
            done.add(url)
    return ret

In [16]:
for cat in concepts:
    urls = read_urls(cat)
    new_urls = uniq_list(urls)
    save_urls(new_urls, cat)

In [13]:
urls = read_urls("birds")

In [14]:
len(urls)

991

In [15]:
len(uniq_list(urls))

989

### Bing, try and error

In [22]:
urls = collect_1100_images("bay")

35
70
105
140
175
210
245
280
315
350
385
420
455
490
525
560
595
630
665
700
735
770
805
840
875
910
945
980
984
reach to last image


In [70]:
lurls = list(urls)

In [71]:
lurls[len(lurls)-1]

'http://upload.wikimedia.org/wikipedia/commons/a/a2/Gull_Island,_Witless_Bay.jpg'

971

In [28]:
top_url = URL_TEMPLATE.format("bay")
driver.get(top_url)


In [29]:
collect_all_urls(130)

70
105
140


{'http://1.bp.blogspot.com/-SgeSYCLNPIU/T3rCgZLEb0I/AAAAAAAAAxk/ocs8TqKvyOU/s1600/Hanauma-Bay.jpg',
 'http://2.bp.blogspot.com/-6wyHoBGwqT8/UVAR_ujR98I/AAAAAAAAYLw/JK8zAA4nt2A/s1600/Horseshoe-Bay-Bermuda.jpg',
 'http://234659.temp-dns.com/wp-content/uploads/2012/04/Mumbles-ViewCCSpJL1.jpg',
 'http://askbeach.com/wp-content/uploads/2015/11/stpauls_bay_2.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/horseshoe_2.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/horseshoe_4.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/shoalbay_3.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/wineglass_4.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/wineglass_6.jpg',
 'http://askbeach.com/wp-content/uploads/2016/05/golden_4.jpg',
 'http://ciaobambino.com/wp-content/uploads/2013/08/little-dix-bay-beach.jpg',
 'http://cornwall-beaches.co.uk/public/photos/daymer-bay.jpg',
 'http://foundtheworld.com/wp-content/uploads/2017/02/The-Bay-of-Islands-1.jpg',
 'http://gardenrou

In [119]:
driver.get(top_url)

In [50]:
atags = driver.find_elements_by_class_name("iusc")

In [51]:
len(atags)

245

In [35]:
scroll_to(atags[len(atags)-1])

In [58]:
driver.save_screenshot('myscreenshot.png')

True

In [44]:
driver.find_element_by_partial_link_text('See more images').click()

In [140]:
last = atags[len(atags)-1]

In [145]:
last.find_element_by_xpath("./../../../..").get_attribute("outerHTML")

'<ul data-row="52" class="dgControl_list " data-infullrow="1"><li data-idx="208" style="width:330px; height: 209px"><div class="iuscp varh" style="width:330px" data-hovstyle="" data-evt="1" gfhk="ngf"><div class="imgpt"><a class="iusc" style="height:209px;width:330px" m="{&quot;cid&quot;:&quot;XCmcIitf&quot;,&quot;purl&quot;:&quot;http://www.lake67.com/australia/new-south-wales/byron-bay/&quot;,&quot;murl&quot;:&quot;http://www.lake67.com/assets/Creative-Commons/Bryon-Bay-NSW.jpg&quot;,&quot;turl&quot;:&quot;http://tse3.mm.bing.net/th?id=OIP.XCmcIitfoJra4bo-DWQdQAEsB7&amp;pid=15.1&quot;,&quot;md5&quot;:&quot;5c299c222b5fa09adae1ba3e0d641d40&quot;}" mad="{&quot;turl&quot;:&quot;http://tse3.mm.bing.net/th?id=OIP.XCmcIitfoJra4bo-DWQdQAEsB7&amp;w=300&amp;h=123&amp;pid=1.1&quot;,&quot;maw&quot;:&quot;300&quot;,&quot;mah&quot;:&quot;123&quot;,&quot;mid&quot;:&quot;F000CF3ACF14C5250CEF78C0056BE99187575773&quot;}" href="/images/search?view=detailV2&amp;ccid=XCmcIitf&amp;id=F000CF3ACF14C5250CEF

In [ ]:
# childElement.findElement(By.xpath("./.."));

In [146]:
last_ul = last.find_element_by_xpath("./../../../..")

In [149]:
div = driver.find_element_by_id('mmComponent_images_1_exp')

In [152]:
div.get_attribute("outerHTML")

'<div id="mmComponent_images_1_exp" class="expandButton disabled" data-expandon=" scroll" data-disablelogging="1"><a href="javascript:void(0);"></a></div>'

In [ ]:
<div id="mmComponent_images_1_exp" class="expandButton active" data-expandon=" scroll" data-disablelogging="1"><a href="javascript:void(0);"></a></div>

In [56]:
driver.execute_script('return window.pageYOffset;')

43271

In [158]:
scroll_to(atags[209])

In [57]:
driver.execute_script('window.scrollBy(0,250)')

In [16]:
atags = driver.find_elements_by_class_name("iusc")

In [17]:
len(atags)

210

In [179]:
driver.get_log('driver')

[]

In [103]:
atags[209].get_attribute("outerHTML")

'<a class="iusc" style="height:189px;width:317px" m="{&quot;cid&quot;:&quot;nHjqty2I&quot;,&quot;purl&quot;:&quot;http://commons.wikimedia.org/wiki/File:Lake_Rotoehu,_Bay_of_Plenty,_New_Zealand,_3_April_2008.jpg&quot;,&quot;murl&quot;:&quot;http://upload.wikimedia.org/wikipedia/commons/3/32/Lake_Rotoehu,_Bay_of_Plenty,_New_Zealand,_3_April_2008.jpg&quot;,&quot;turl&quot;:&quot;http://tse2.mm.bing.net/th?id=OIP.nHjqty2IUusOI3X1G2_Z8wEsC9&amp;pid=15.1&quot;,&quot;md5&quot;:&quot;9c78eab72d8852eb0e2375f51b6fd9f3&quot;}" mad="{&quot;turl&quot;:&quot;http://tse4.mm.bing.net/th?id=OIP.nHjqty2IUusOI3X1G2_Z8wEsC9&amp;w=300&amp;h=189&amp;pid=1.1&quot;,&quot;maw&quot;:&quot;300&quot;,&quot;mah&quot;:&quot;189&quot;,&quot;mid&quot;:&quot;A91296BDB86603445B6903AF40DEEC54EA49FF21&quot;}" href="/images/search?view=detailV2&amp;ccid=nHjqty2I&amp;id=A91296BDB86603445B6903AF40DEEC54EA49FF21&amp;thid=OIP.nHjqty2IUusOI3X1G2_Z8wEsC9&amp;q=bay&amp;simid=608009896000292481&amp;selectedIndex=218" h="ID=image

In [115]:
atags[209].get_attribute("outerHTML")

'<a class="iusc" style="height:189px;width:317px" m="{&quot;cid&quot;:&quot;nHjqty2I&quot;,&quot;purl&quot;:&quot;http://commons.wikimedia.org/wiki/File:Lake_Rotoehu,_Bay_of_Plenty,_New_Zealand,_3_April_2008.jpg&quot;,&quot;murl&quot;:&quot;http://upload.wikimedia.org/wikipedia/commons/3/32/Lake_Rotoehu,_Bay_of_Plenty,_New_Zealand,_3_April_2008.jpg&quot;,&quot;turl&quot;:&quot;http://tse2.mm.bing.net/th?id=OIP.nHjqty2IUusOI3X1G2_Z8wEsC9&amp;pid=15.1&quot;,&quot;md5&quot;:&quot;9c78eab72d8852eb0e2375f51b6fd9f3&quot;}" mad="{&quot;turl&quot;:&quot;http://tse4.mm.bing.net/th?id=OIP.nHjqty2IUusOI3X1G2_Z8wEsC9&amp;w=300&amp;h=189&amp;pid=1.1&quot;,&quot;maw&quot;:&quot;300&quot;,&quot;mah&quot;:&quot;189&quot;,&quot;mid&quot;:&quot;A91296BDB86603445B6903AF40DEEC54EA49FF21&quot;}" href="/images/search?view=detailV2&amp;ccid=nHjqty2I&amp;id=A91296BDB86603445B6903AF40DEEC54EA49FF21&amp;thid=OIP.nHjqty2IUusOI3X1G2_Z8wEsC9&amp;q=bay&amp;simid=608009896000292481&amp;selectedIndex=218" h="ID=image

In [112]:
scroll_to(atags[208])

In [19]:
top_url = URL_TEMPLATE.format("bay")

In [83]:
driver.get(top_url)

In [89]:
collect_all_urls()

{'http://1.bp.blogspot.com/-SgeSYCLNPIU/T3rCgZLEb0I/AAAAAAAAAxk/ocs8TqKvyOU/s1600/Hanauma-Bay.jpg',
 'http://2.bp.blogspot.com/-6wyHoBGwqT8/UVAR_ujR98I/AAAAAAAAYLw/JK8zAA4nt2A/s1600/Horseshoe-Bay-Bermuda.jpg',
 'http://234659.temp-dns.com/wp-content/uploads/2012/04/Mumbles-ViewCCSpJL1.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/horseshoe_2.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/horseshoe_4.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/wineglass_4.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/wineglass_6.jpg',
 'http://ciaobambino.com/wp-content/uploads/2013/08/little-dix-bay-beach.jpg',
 'http://foundtheworld.com/wp-content/uploads/2017/02/The-Bay-of-Islands-1.jpg',
 'http://gardenroutebliss.co.za/wp-content/uploads/2016/07/Plettenberg-Bay-Vacation-Home.jpg',
 'http://hanifworld.com/Jamaica/22-Montego%20Bay%20beach.JPG',
 'http://holidayhomes.ca/wp-content/uploads/2016/04/Image-1-1.jpg',
 'http://i4.dailypost.co.uk/incoming/article2509705.ec

In [90]:
len(Out[89])

103

In [116]:
driver.close()

In [21]:
atags = driver.find_elements_by_class_name("iusc")

In [23]:
atags[0].get_attribute("m")

'{"cid":"5Su3/66B","purl":"https://achillfieldschool.wordpress.com/2014/05/08/a-visit-to-keem-bay/","murl":"https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg","turl":"http://tse4.mm.bing.net/th?id=OIP.5Su3_66BqHzeAhUAbUgklQEsDI&pid=15.1","md5":"e52bb7ffae81a87cde0215006d482495"}'

In [26]:
json.loads(atags[0].get_attribute("m"))['murl']

'https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg'

In [27]:
len(atags)

35

In [28]:
json.loads(atags[34].get_attribute("m"))['murl']

'http://www.wyndhamap.com/wps/wcm/connect/0b16df8f-5790-4943-b378-acaac7c9d563/Shoal-Bay-Resort-Banners2-6.jpg?MOD=AJPERES&CACHEID=0b16df8f-5790-4943-b378-acaac7c9d563'

In [29]:
atags[34].get_attribute("outerHTML")

'<a class="iusc" style="height:160px;width:299px" m="{&quot;cid&quot;:&quot;G60mqYnO&quot;,&quot;purl&quot;:&quot;http://www.wyndhamap.com/wps/wcm/connect/wyndham/home/resorts/australia/nsw/ramada-shoal-bay/overview/ramada-shoal-bay?loc=whg&quot;,&quot;murl&quot;:&quot;http://www.wyndhamap.com/wps/wcm/connect/0b16df8f-5790-4943-b378-acaac7c9d563/Shoal-Bay-Resort-Banners2-6.jpg?MOD=AJPERES&amp;CACHEID=0b16df8f-5790-4943-b378-acaac7c9d563&quot;,&quot;turl&quot;:&quot;http://tse1.mm.bing.net/th?id=OIP.G60mqYnOWbE-DA7VLEfBugEsB1&amp;pid=15.1&quot;,&quot;md5&quot;:&quot;1bad26a989ce59b13e0c0ed52c47c1ba&quot;}" mad="{&quot;turl&quot;:&quot;http://tse1.mm.bing.net/th?id=OIP.G60mqYnOWbE-DA7VLEfBugEsB1&amp;w=299&amp;h=117&amp;pid=1.1&quot;,&quot;maw&quot;:&quot;299&quot;,&quot;mah&quot;:&quot;117&quot;,&quot;mid&quot;:&quot;CA52D6F4DD2EEFEC1E9FE0390BCA9EDE1D7CA5D6&quot;}" href="/images/search?view=detailV2&amp;ccid=G60mqYnO&amp;id=CA52D6F4DD2EEFEC1E9FE0390BCA9EDE1D7CA5D6&amp;thid=OIP.G60mqYnOWb

In [30]:
atags[0].get_attribute("outerHTML")

'<a class="iusc" style="height:160px;width:242px" m="{&quot;cid&quot;:&quot;5Su3/66B&quot;,&quot;purl&quot;:&quot;https://achillfieldschool.wordpress.com/2014/05/08/a-visit-to-keem-bay/&quot;,&quot;murl&quot;:&quot;https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg&quot;,&quot;turl&quot;:&quot;http://tse4.mm.bing.net/th?id=OIP.5Su3_66BqHzeAhUAbUgklQEsDI&amp;pid=15.1&quot;,&quot;md5&quot;:&quot;e52bb7ffae81a87cde0215006d482495&quot;}" mad="{&quot;turl&quot;:&quot;http://tse2.mm.bing.net/th?id=OIP.5Su3_66BqHzeAhUAbUgklQEsDI&amp;w=300&amp;h=200&amp;pid=1.1&quot;,&quot;maw&quot;:&quot;300&quot;,&quot;mah&quot;:&quot;200&quot;,&quot;mid&quot;:&quot;DD089B779F640C1442AB515DB782A4987B7378EB&quot;}" href="/images/search?view=detailV2&amp;ccid=5Su3%2f66B&amp;id=DD089B779F640C1442AB515DB782A4987B7378EB&amp;thid=OIP.5Su3_66BqHzeAhUAbUgklQEsDI&amp;q=bay&amp;simid=608003462168708023&amp;selectedIndex=0" h="ID=images,5169.1" data-focevt="1"><div class="img_cont hoff"><img class="mimg

In [31]:
ActionChains(driver).move_to_element(atags[34]).perform()

In [32]:
atags2 = driver.find_elements_by_class_name("iusc")

In [33]:
len(atags2)

70

In [34]:
atags2[0].get_attribute("outerHTML")

'<a class="iusc" style="height:160px;width:242px" m="{&quot;cid&quot;:&quot;5Su3/66B&quot;,&quot;purl&quot;:&quot;https://achillfieldschool.wordpress.com/2014/05/08/a-visit-to-keem-bay/&quot;,&quot;murl&quot;:&quot;https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg&quot;,&quot;turl&quot;:&quot;http://tse4.mm.bing.net/th?id=OIP.5Su3_66BqHzeAhUAbUgklQEsDI&amp;pid=15.1&quot;,&quot;md5&quot;:&quot;e52bb7ffae81a87cde0215006d482495&quot;}" mad="{&quot;turl&quot;:&quot;http://tse2.mm.bing.net/th?id=OIP.5Su3_66BqHzeAhUAbUgklQEsDI&amp;w=300&amp;h=200&amp;pid=1.1&quot;,&quot;maw&quot;:&quot;300&quot;,&quot;mah&quot;:&quot;200&quot;,&quot;mid&quot;:&quot;DD089B779F640C1442AB515DB782A4987B7378EB&quot;}" href="/images/search?view=detailV2&amp;ccid=5Su3%2f66B&amp;id=DD089B779F640C1442AB515DB782A4987B7378EB&amp;thid=OIP.5Su3_66BqHzeAhUAbUgklQEsDI&amp;q=bay&amp;simid=608003462168708023&amp;selectedIndex=0" h="ID=images,5169.1" data-focevt="1"><div class="img_cont hoff"><img class="mimg

In [35]:
atags2[34].get_attribute("outerHTML")

'<a class="iusc" style="height:160px;width:299px" m="{&quot;cid&quot;:&quot;G60mqYnO&quot;,&quot;purl&quot;:&quot;http://www.wyndhamap.com/wps/wcm/connect/wyndham/home/resorts/australia/nsw/ramada-shoal-bay/overview/ramada-shoal-bay?loc=whg&quot;,&quot;murl&quot;:&quot;http://www.wyndhamap.com/wps/wcm/connect/0b16df8f-5790-4943-b378-acaac7c9d563/Shoal-Bay-Resort-Banners2-6.jpg?MOD=AJPERES&amp;CACHEID=0b16df8f-5790-4943-b378-acaac7c9d563&quot;,&quot;turl&quot;:&quot;http://tse1.mm.bing.net/th?id=OIP.G60mqYnOWbE-DA7VLEfBugEsB1&amp;pid=15.1&quot;,&quot;md5&quot;:&quot;1bad26a989ce59b13e0c0ed52c47c1ba&quot;}" mad="{&quot;turl&quot;:&quot;http://tse1.mm.bing.net/th?id=OIP.G60mqYnOWbE-DA7VLEfBugEsB1&amp;w=299&amp;h=117&amp;pid=1.1&quot;,&quot;maw&quot;:&quot;299&quot;,&quot;mah&quot;:&quot;117&quot;,&quot;mid&quot;:&quot;CA52D6F4DD2EEFEC1E9FE0390BCA9EDE1D7CA5D6&quot;}" href="/images/search?view=detailV2&amp;ccid=G60mqYnO&amp;id=CA52D6F4DD2EEFEC1E9FE0390BCA9EDE1D7CA5D6&amp;thid=OIP.G60mqYnOWb

In [36]:
atags2[69].get_attribute("outerHTML")

'<a class="iusc" style="height:167px;width:252px" m="{&quot;cid&quot;:&quot;TJWfXfcv&quot;,&quot;purl&quot;:&quot;http://www.australiascoralcoast.com/destinations/cervantes-jurien-bay-region/jurien-bay&quot;,&quot;murl&quot;:&quot;http://www.australiascoralcoast.com/sfimages/default-source/jurien-bay/jurien-bay-and-surrounds---jurien-bay-marine-park-small.jpg?size=294&quot;,&quot;turl&quot;:&quot;http://tse1.mm.bing.net/th?id=OIP.TJWfXfcvcSFdBlpJAcB_SgEsDH&amp;pid=15.1&quot;,&quot;md5&quot;:&quot;4c959f5df72f71215d065a4901c07f4a&quot;}" mad="{&quot;turl&quot;:&quot;http://tse2.mm.bing.net/th?id=OIP.TJWfXfcvcSFdBlpJAcB_SgEsDH&amp;w=300&amp;h=199&amp;pid=1.1&quot;,&quot;maw&quot;:&quot;300&quot;,&quot;mah&quot;:&quot;199&quot;,&quot;mid&quot;:&quot;15A13BB2BF6448F28F9DD043E14F8B04916E590B&quot;}" href="/images/search?view=detailV2&amp;ccid=TJWfXfcv&amp;id=15A13BB2BF6448F28F9DD043E14F8B04916E590B&amp;thid=OIP.TJWfXfcvcSFdBlpJAcB_SgEsDH&amp;q=bay&amp;simid=608030159678276020&amp;selectedIn

In [42]:
hval = 'ID=images,5373.1'

In [69]:
set() | {1, 2, 3}

{1, 2, 3}

In [56]:
get_urls(atags2)

('ID=images.6000_2,5228.1',
 ['https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg',
  'http://traveleering.com/wp-content/uploads/2017/02/8.Wineglass-bay-Tasmania.jpg',
  'https://media-cdn.tripadvisor.com/media/photo-s/01/c5/d9/3f/view-of-the-bay-from.jpg',
  'http://tassiegrammer.com.au/wp-content/uploads/2016/01/WineglassBay3.jpg',
  'http://www.australiantraveller.com/wp-content/uploads/2006/04/100-Things-To-Do-Before-You-Die-064-Wineglass-Bay.jpg',
  'http://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg',
  'http://ciaobambino.com/wp-content/uploads/2013/08/little-dix-bay-beach.jpg',
  'https://www.engelvoelkers.com/wp-content/uploads/2015/05/Clearwater_Bay.jpg',
  'https://upload.wikimedia.org/wikipedia/commons/a/a2/Horseshoe_Bay_Stevage.jpg',
  'http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg',
  'http://mustseeplaces.eu/wp-content/uploads/2016/01/freycinet-national-park-tasmania-australia.jpg',
  'http://media-c

In [57]:
get_urls(atags2, hval)

('ID=images.6000_2,5228.1',
 ['http://upload.wikimedia.org/wikipedia/commons/3/3f/Durness_Bay.JPG',
  'http://askbeach.com/wp-content/uploads/2016/01/wineglass_4.jpg',
  'https://www.cornwall-beaches.co.uk/public/photos/lantic-bay-beach.jpg',
  'http://upload.wikimedia.org/wikipedia/commons/8/83/Cardigan_bay_graham_well.jpg',
  'http://www.hermitagebay.com/content/desktop/img/backgrounds/img_home_windsurfing.jpg',
  'http://gardenroutebliss.co.za/wp-content/uploads/2016/07/Plettenberg-Bay-Vacation-Home.jpg',
  'https://www.pgl.co.uk/Files/Files/Schools/Primary%20Schools/Centre%20Gallery/Osmington%20Bay/PS-G-Centre-Osmington-Bay-sea-beach-durdle-door-jurassic-coast-for-Primary-Schools.jpg',
  'http://www.doc.govt.nz/global/images/places/northland/maitai-bay/maitai-bay-karikari-peninsula-gallery.jpg',
  'http://foundtheworld.com/wp-content/uploads/2017/02/The-Bay-of-Islands-1.jpg',
  'http://upload.wikimedia.org/wikipedia/commons/9/9a/Watsons_Bay_-_Camp_Cove_Beach,_Sydney_2_-_Nov_2008.jp

In [47]:
list(range(3, 5))

[3, 4]

In [40]:
atags2[34].get_attribute('h') == 'ID=images,5373.1'

True

In [43]:
find_index(atags2, hval)

34

In [58]:
_, urls = Out[56]

In [59]:
len(urls)

70

In [62]:
len(sorted(set(urls)))

68

In [63]:
len(set(urls))

68

In [64]:
S = {1, 2, 3, 4}
T = {3, 4, 5, 6}

In [4]:
# selenium test.

# driver.get('http://www.google.com')
# print(driver.title)
# driver.close()

### yahoo jp trial (not work)

In [1]:
URL_TEMPLATE = "https://search.yahoo.co.jp/image/search?p={}"

In [2]:
top_url = URL_TEMPLATE.format("bay")

In [10]:
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="/usr/local/bin/chromedriver")
driver.maximize_window()

In [11]:
driver.get(top_url)

In [12]:
atags= driver.find_elements_by_css_selector("a[target='imagewin']")

In [13]:
atags

[<selenium.webdriver.remote.webelement.WebElement (session="e91829f1fb87ebbe6dd09e26de8e1bb9", element="0.558469916898801-1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e91829f1fb87ebbe6dd09e26de8e1bb9", element="0.558469916898801-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e91829f1fb87ebbe6dd09e26de8e1bb9", element="0.558469916898801-3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e91829f1fb87ebbe6dd09e26de8e1bb9", element="0.558469916898801-4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e91829f1fb87ebbe6dd09e26de8e1bb9", element="0.558469916898801-5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e91829f1fb87ebbe6dd09e26de8e1bb9", element="0.558469916898801-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e91829f1fb87ebbe6dd09e26de8e1bb9", element="0.558469916898801-7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e91829f1fb87ebbe6dd09e26de8e1bb9", element="

# Yahoo Image search trial

This page often cause page crash. I don't know the reason.

In [13]:
top_url = "http://images.search.yahoo.com/"

In [14]:
def search_word(word):
    driver.get(top_url)
    input = driver.find_element_by_id('yschsp')
    input.send_keys(word)
    btn = driver.find_element_by_class_name('ygbt')
    btn.click()

In [50]:
# driver.title

'bay - Yahoo Image Search Results'

In [15]:
import time
def ctrl_click_to_newtab(link):
    for retry in range(5):
        ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()
        if len(driver.window_handles) == 2:
            return
        time.sleep(0.5)
    raise Exception("do not open new tab")


In [16]:
def get_image_url_from_viewimage():
    link = driver.find_element_by_link_text('View Image')
    ctrl_click_to_newtab(link)
    driver.switch_to_window(driver.window_handles[1])
    time.sleep(1)
    url = driver.current_url
    driver.close()
    time.sleep(0.5)
    driver.switch_to_window(driver.window_handles[0])
    time.sleep(0.5)
    return url

In [17]:
def get_image_url_from_iholder():
    div = driver.find_element_by_class_name('iholder')
    img = div.find_element_by_id('img')
    return img.get_attribute('src')

In [18]:
def get_image_url(image_li):
    image_li.click()
    time.sleep(0.5)
    return get_image_url_from_viewimage()

In [19]:
def click_more():
    more = driver.find_element_by_name('more-res')
    more.click()

In [20]:
def find_index_by_id(all_li, id):
    for i in range(len(all_li)):
        if(all_li[i].get_attribute('id') == id):
            return i
    return -1

In [21]:
def click_next():
    driver.find_element_by_css_selector(".nav.right").click()

In [68]:
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="/usr/local/bin/chromedriver")
driver.maximize_window()

search_word('bay')
first_res = driver.find_element_by_id("resitem-0")
par = first_res.parent
all_lis = par.find_elements_by_tag_name('li')

In [25]:
from selenium.common.exceptions import StaleElementReferenceException, WebDriverException
def get_all_url(all_li, from_count=0, limit=20):
    res = []
    lastid = None
    begin = find_index_by_id(all_li, "resitem-{}".format(from_count))
    
    ActionChains(driver).move_to_element(all_li[begin]).perform()
    time.sleep(1)
    
    all_li[begin].click()
    time.sleep(0.5)
    count = from_count
    while count < limit:
        try:
            print(count)
            res.append(get_image_url_from_iholder())
        except WebDriverException:
            print("WebDriverException: Please restart from here.")
            return(count, res)
        click_next()
        time.sleep(2)
        if (count%5) == 0:
            time.sleep(3)
        if (count%25) == 0:
            time.sleep(5)
        count +=1
    return (count, res)

In [26]:
count, urls = get_all_url(all_lis, limit=100)

0


WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: headless chrome=62.0.3202.75)
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [71]:
get_image_url_from_iholder()

'http://images.trvl-media.com/media/content/shared/images/travelguides/Bay-of-Islands-6024124-smallTabletRetina.jpg'

In [72]:
click_next()

In [73]:
get_image_url_from_iholder()

WebDriverException: Message: unknown error: session deleted because of page crash
from tab crashed
  (Session info: headless chrome=62.0.3202.75)
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [22]:
count, urls = get_all_url(all_lis, limit=100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
WebDriverException: Please restart from here.


In [23]:
urls

['https://tse1.mm.bing.net/th?id=OIP.5Su3_66BqHzeAhUAbUgklQEsDI&pid=15.1&P=0&w=259&h=174',
 'http://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg',
 'http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg',
 'http://ciaobambino.com/wp-content/uploads/2013/08/little-dix-bay-beach.jpg',
 'http://www.worldofwanderlust.com/wp-content/uploads/2015/07/P1010074.jpg',
 'https://media-cdn.tripadvisor.com/media/photo-s/01/c5/d9/3f/view-of-the-bay-from.jpg',
 'http://mustseeplaces.eu/wp-content/uploads/2016/01/freycinet-national-park-tasmania-australia.jpg',
 'http://upload.wikimedia.org/wikipedia/commons/6/6f/St_oswalds_bay_near_lulworth_dorset_arp.jpg',
 'http://upload.wikimedia.org/wikipedia/commons/c/ce/Wineglass_Bay_from_Lookout_crop.jpg',
 'https://www.engelvoelkers.com/wp-content/uploads/2015/05/Clearwater_Bay.jpg',
 'http://static.thousandwonders.net/Hanauma.Bay.original.15812.jpg',
 'http://internet-travelgroups.com/wp-content/uploads/2016/01/W

In [23]:
count

21

In [22]:
count, urls = get_all_url(all_lis, from_count=21, limit=100)

0
1
2
3
4
5
6
WebDriverException: Please restart from here.


In [22]:
count, urls = get_all_url(all_lis, from_count=22, limit=100)

22
23
24
25
26
27
WebDriverException: Please restart from here.


In [23]:
count

27

In [24]:
urls

['https://tse3.mm.bing.net/th?id=OIP.QZT2x02R51ZdgjFCwnY6BwEgDY&pid=15.1&P=0&w=220&h=165',
 'http://askbeach.com/wp-content/uploads/2016/01/wineglass_6.jpg',
 'http://www.pgl.co.uk/Files/Files/Schools/Primary%20Schools/Centre%20Gallery/Osmington%20Bay/PS-G-Centre-Osmington-Bay-sea-beach-durdle-door-jurassic-coast-for-Primary-Schools.jpg',
 'https://tse4.mm.bing.net/th?id=OIP._ps0dCwgM-_IO4R9q-jC3wEsC7&pid=15.1&P=0&w=274&h=172',
 'http://www.100besteverything.com/best-beaches/i-53-1924_hanaumabaywp1.jpg']

In [22]:
count, urls = get_all_url(all_lis, from_count=27, limit=100)

27
28
WebDriverException: Please restart from here.


In [23]:
urls

['https://tse3.mm.bing.net/th?id=OIP.wcqIe65uyBMM1hW40t2QpQEsDI&pid=15.1&P=0&w=257&h=173']

In [20]:
count, urls = get_all_url(all_lis, from_count=28, limit=100)

28
29
30
31
32
WebDriverException: Please restart from here.


In [21]:
urls

['https://tse3.mm.bing.net/th?id=OIP.yINJMexEXGs7QoDx6KsGzgEsDo&pid=15.1&P=0&w=220&h=171',
 'http://www.touristsbook.com/wp-content/uploads/2016/01/wineglass-bay-tasmnia.jpg',
 'http://1.bp.blogspot.com/-mfYgwpoi5-0/UOQ98ZrCJcI/AAAAAAAAAts/Y1pXc0pMcAY/s1600/Byron+Bay+View.jpg',
 'https://www.cornwall-beaches.co.uk/public/photos/lantic-bay-beach.jpg']

In [20]:
count, urls = get_all_url(all_lis, from_count=32, limit=100)

32
33
34
35
36
WebDriverException: Please restart from here.


In [21]:
urls

['https://tse3.mm.bing.net/th?id=OIP.8YpLRoCzCJNYR3eC0jTd5wEsDI&pid=15.1&P=0&w=246&h=165',
 'https://tse4.mm.bing.net/th?id=OIP.uyrbgx3_iVSAMugF3F1tVgE1DK&pid=15.1&P=0&w=319&h=209',
 'http://images.trvl-media.com/media/content/shared/images/travelguides/Byron-Bay-and-vicinity-6138873-smallTabletRetina.jpg',
 'http://www.doc.govt.nz/global/images/places/northland/maitai-bay/maitai-bay-karikari-peninsula-gallery.jpg']

In [20]:
count, urls = get_all_url(all_lis, from_count=36, limit=100)

36
37
38
WebDriverException: Please restart from here.


In [21]:
urls

['https://tse3.mm.bing.net/th?id=OIP.TCQAnyFjNmo-37TVobOX1gEsB1&pid=15.1&P=0&w=368&h=144',
 'http://www.carlisle-bay.com/images/carlisle-bay-hotel_antigua-compressed.jpg']

In [20]:
count, urls = get_all_url(all_lis, from_count=38, limit=100)

38
39
WebDriverException: Please restart from here.


In [21]:
urls

['https://tse4.mm.bing.net/th?id=OIP.ijApGq_tsLGpakNyR7X9GwEsBd&pid=15.1&P=0&w=368&h=115']

In [20]:
count, urls = get_all_url(all_lis, from_count=39, limit=100)

39
40
41
42
43
44
45
46
WebDriverException: Please restart from here.


In [22]:
urls

['https://tse1.mm.bing.net/th?id=OIP.QWVv30E5K-M_78aWQvlaWQEsDI&pid=15.1&P=0&w=256&h=172',
 'https://www.leisuretours.co.nz/wp-content/uploads/Bay-of-Islands.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Pillar_Point_and_Half_Moon_Bay.jpg/1200px-Pillar_Point_and_Half_Moon_Bay.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/d/d2/Clear_Water_Bay_1st_Beach.jpg',
 'https://media-cdn.tripadvisor.com/media/photo-o/0d/d2/f0/3a/plantation-bay-resort.jpg',
 'https://d19lgisewk9l6l.cloudfront.net/wexas/www/images/largeimages/destinations/Antigua-and-Barbuda/Carlisle_Bay.jpg',
 'http://askbeach.com/wp-content/uploads/2016/01/horseshoe_2.jpg']

In [19]:
get_image_url(all_lis[106])

'https://www.iod.org.nz/portals/0/Branches%20and%20events/BayofPlenty_header.jpg'

In [20]:
get_image_url(all_lis[107])

'http://www.tourstas.com.au/wp-content/uploads/2014/05/Wineglass_Bay_2_940x350.jpg'

In [21]:
get_image_url(all_lis[108])

'http://tassiegrammer.com.au/wp-content/uploads/2016/01/WineglassBay3.jpg'

In [22]:
get_image_url(all_lis[109])

WebDriverException: Message: unknown error: session deleted because of page crash
from tab crashed
  (Session info: headless chrome=62.0.3202.75)
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [ ]:
get_image_url(all_lis[110])

In [14]:
list(map(lambda el: el.get_attribute("id"), all_lis))

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'pivot_more',
 '',
 '',
 '',
 '',
 '',
 '',
 'yss',
 '',
 '',
 'license',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'resitem-fpub0',
 'resitem-fpub1',
 'resitem-fpub2',
 'resitem-fpub3',
 'resitem-fpub4',
 'resitem-fpub5',
 'resitem-fpub6',
 'resitem-fpub7',
 'resitem-fpub8',
 'resitem-fpub9',
 'resitem-fpub10',
 'resitem-fpub11',
 'resitem-fpub12',
 'resitem-fpub13',
 'resitem-fpub14',
 'resitem-fpub15',
 'resitem-fpub16',
 'resitem-fpub17',
 'resitem-fpub18',
 'resitem-fpub19',
 '',
 'resitem-0',
 'resitem-1',
 'resitem-2',
 'resitem-3',
 'resitem-4',
 'resitem-5',
 'resitem-6',
 'resitem-7',
 'resitem-8',
 'resitem-9',
 'resitem-10',
 'resitem-11',
 'resitem-12',
 'resitem-13',
 'resitem-14',
 'resitem-15',
 'resitem-16',
 'resitem-17',
 'resitem-

In [16]:
first_res.click()

In [17]:
driver.find_element_by_class_name('iholder')

<selenium.webdriver.remote.webelement.WebElement (session="48c95370da3fbfad83f4ade1687ce9f0", element="0.5663800474746361-160")>

In [18]:
div = driver.find_element_by_class_name('iholder')

In [19]:
div.find_element_by_id('img')

<selenium.webdriver.remote.webelement.WebElement (session="48c95370da3fbfad83f4ade1687ce9f0", element="0.5663800474746361-161")>

In [20]:
img = div.find_element_by_id('img')

In [21]:
img.get_attribute('src')

'https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg'

In [22]:
get_image_url_from_viewimage()

'https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg'

In [29]:
click_next()

In [30]:
get_image_url_from_iholder()

'http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg'

In [24]:
(20%10) == 0

True

In [20]:
all_lis[88].get_attribute('id')

'resitem-0'

In [26]:
get_image_url(all_lis[88])

'https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg'

In [27]:
get_image_url(all_lis[89])

'https://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg'

In [29]:
get_image_url(all_lis[90]), get_image_url(all_lis[91]), get_image_url(all_lis[92]), get_image_url(all_lis[93])

('http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg',
 'http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg',
 'http://www.worldofwanderlust.com/wp-content/uploads/2015/07/P1010074.jpg',
 'https://media-cdn.tripadvisor.com/media/photo-s/01/c5/d9/3f/view-of-the-bay-from.jpg')

In [25]:
all_lis[140].get_attribute('id')

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=62.0.3202.75)
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [38]:
all_lis[89].click()

In [23]:
link = driver.find_element_by_link_text('View Image')

In [24]:
link

<selenium.webdriver.remote.webelement.WebElement (session="4a6d7e84a7ea8ce3d878bd29253b6b63", element="0.34873234290672506-160")>

In [20]:
all_lis[92].click()

In [21]:
link = driver.find_element_by_link_text('View Image')
ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()

In [40]:
driver.window_handles

WebDriverException: Message: no such session
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [120]:
all_lis2[143].get_attribute("id").startswith('resitem')

True

In [129]:
all_lis2[93].get_attribute('innerHTML')

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=62.0.3202.75)
  (Driver info: chromedriver=2.33.506092 (733a02544d189eeb751fe0d7ddca79a0ee28cce4),platform=Linux 4.9.0-4-amd64 x86_64)


In [31]:
first_res.click()

In [109]:
find_index_by_id(all_lis2, 'resitem-49')

139

In [125]:
find_index_by_id(all_lis2, 'resitem-0')

88

In [ ]:
#from time import sleep
#sleep(0.05)

In [68]:
def find_index(list, target):
    for i in range(len(list)):
        if(list[i].id == target.id):
            return i
    return -1

In [69]:
find_index(all_lis, first_res)

88

In [83]:
len(all_lis)

159

In [71]:
all_lis[88].get_attribute("id")

'resitem-0'

In [74]:
all_lis[141].get_attribute("id")

'resitem-53'

In [75]:
all_lis[142].get_attribute("id")

'yui_3_5_1_1_1509360357945_752'

In [87]:
all_lis[150].get_attribute("id")

'yui_3_5_1_1_1509360357945_784'

In [33]:
link = driver.find_element_by_link_text('View Image')

In [37]:
ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()

In [38]:
driver.window_handles

['CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f',
 'CDwindow-0e5a14ab-ae1b-4ce7-8b61-5771e4070e18']

In [42]:
driver.current_window_handle

'CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f'

In [44]:
driver.switch_to_window(driver.window_handles[1])

In [45]:
driver.current_url

'https://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg'

In [46]:
urllist = []

In [47]:
urllist.append(driver.current_url)

In [48]:
driver.close()

In [49]:
driver.window_handles

['CDwindow-e0a609a0-3c4e-48fa-9d75-3e428a57205f']

In [51]:
driver.switch_to_window(driver.window_handles[0])

In [52]:
driver.current_url

'http://images.search.yahoo.com/search/images;_ylt=AwrTcXiu9fZZ1JcAkweLuLkF;_ylc=X1MDOTYwNTc0ODMEX3IDMgRiY2sDZTZycGdwNWN2ZHRkZSUyNmIlM0QzJTI2cyUzRHEyBGZyAwRncHJpZANSMWd6aFl0eFE5dUtKbk1KM2V6dVJBBG10ZXN0aWQDbnVsbARuX3N1Z2cDMTAEb3JpZ2luA2ltYWdlcy5zZWFyY2gueWFob28uY29tBHBvcwMwBHBxc3RyAwRwcXN0cmwDBHFzdHJsAzMEcXVlcnkDYmF5BHRfc3RtcAMxNTA5MzU2OTc4BHZ0ZXN0aWQDbnVsbA--?gprid=R1gzhYtxQ9uKJnMJ3ezuRA&pvid=A1d10TEwLjJUhulTd3DH.AAyMTA0LgAAAACOSmPr&fr2=sb-top-images.search.yahoo.com&p=bay&ei=UTF-8&iscqry=&fr=sfp#id=0&iurl=http%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2F9%2F9d%2FSmitswinkel_Bay.jpg&action=click'

In [77]:
get_image_url(first_res)

'https://achillfieldschool.files.wordpress.com/2014/05/dsc_0287.jpg'

In [78]:
driver.window_handles

['CDwindow-46fad65f-8a9e-4fcd-9bb0-2e859f5bd4c5']

In [79]:
get_image_url(all_lis[89])

'https://upload.wikimedia.org/wikipedia/commons/9/9d/Smitswinkel_Bay.jpg'

In [80]:
get_image_url(all_lis[90])

'http://www.worldfortravel.com/wp-content/uploads/2013/04/Freycinet-Australia.jpg'

In [81]:
driver.window_handles

['CDwindow-46fad65f-8a9e-4fcd-9bb0-2e859f5bd4c5']

In [115]:
lis = [1, 2, 3]

In [116]:
lis.extend([4, 5, 6])

In [117]:
lis

[1, 2, 3, 4, 5, 6]

In [92]:
all_lis2 = par.find_elements_by_tag_name('li')

In [93]:
len(all_lis2)

199

In [99]:
all_lis2[145].get_attribute("id")

'resitem-58'

In [ ]:
driver.close()

### FireFox trial (not working)

In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

In [3]:
binary = FirefoxBinary('/usr/bin/firefox')
binary.add_command_line_options('-headless')
driver = webdriver.Firefox(executable_path="/usr/local/bin/geckodriver",firefox_binary=binary)

WebDriverException: Message: Process unexpectedly closed with status: 1


In [ ]:
"""
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("disable-infobars")
chrome_options.add_argument("--disable-extensions")
"""